In [1]:
import copy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
2+2

4

In [2]:
from pylab import *
import matplotlib.pyplot as plt
import os
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

print('imported')

bn=gum.BayesNet('frontdoor')
U,X,Z,Y=[ bn.add(name, 2) for name in "UXZY" ] 
print (bn)

all_nodes=['U', 'X','Z', 'Y'] 

bn.addArc(U,X)
bn.addArc(U,Y)
bn.addArc(X,Z)
bn.addArc(Z,Y)
gnb.flow.row(bn)

bn

imported
BN{nodes: 4, arcs: 0, domainSize: 16, dim: 4, mem: 64o}


(pyAgrum.BayesNet<double>@0x55906cde69c0) BN{nodes: 4, arcs: 4, domainSize: 16, dim: 9, mem: 144o}

In [3]:
bn.generateCPT('U')
bn.generateCPT('X')
bn.generateCPT('Z')
bn.generateCPT('Y')

ie=gum.LazyPropagation(bn)
ie.makeInference()
ie.evidenceJointImpact(['X', 'Z', 'Y'], [])

(pyAgrum.Potential<double>@0x55906cd26df0) 
             ||  Y                |
X     |Z     ||0        |1        |
------|------||---------|---------|
0     |0     || 0.0253  | 0.0737  |
1     |0     || 0.0862  | 0.4097  |
0     |1     || 0.1020  | 0.0945  |
1     |1     || 0.1080  | 0.1006  |

In [4]:
# intervened graph         
def get_bn(bn2, intervened):
    for var in intervened:
        for parent in bn2.cpt(var).var_names:
            if parent != var:
                bn2.eraseArc(*(parent, var))


        lst = [0 for i in range(2)]
        lst[intervened[var]] = 1
        print(lst)
        bn2.cpt(var).fillWith(lst)

    return bn2

# Ground True intervention     
intervened={'X':1}
bn_intv=gum.BayesNet(bn)
bn_intv= get_bn(bn_intv, intervened)

all_nodes=['U', 'Z','X', 'Y'] 
ie = gum.LazyPropagation(bn_intv)
var_set = set(all_nodes)
ie.addJointTarget(var_set)
ie.makeInference()

pY_dox= ie.evidenceJointImpact(['Y'],[])
gnb.flow.row(pY_dox, captions=[f'P(Y|do{intervened})'])

[0, 1]


/local/scratch/a/rahman89/PycharmProjects/common-env/venv/lib/python3.10/site-packages/pyAgrum/pyAgrum.py:10018: UserWarning: 
** pyAgrum.Potential.var_names is obsolete in pyAgrum>0.22.9. Please use pyAgrum.Potential.names.

  warnings.warn("\n** pyAgrum.Potential.var_names is obsolete in pyAgrum>0.22.9. Please use pyAgrum.Potential.names.\n")


In [5]:
true_doX_cpt= pY_dox

In [6]:
true_doX={}
for x in true_doX_cpt.loopIn():
    # print(x.todict(), true_doX_cpt[x])
    true_doX[tuple(x.todict().values())]= true_doX_cpt[x]

true_doX

{(0,): 0.29286194666823534, (1,): 0.7071380533317647}

# Data generation

In [7]:
g=gum.BNDatabaseGenerator(bn)
N= 10000
g.drawSamples(N,{})
dataset= g.to_pandas()



dataset = np.array(dataset).astype(int)

dataset= dataset[:, 1:]  #ignoring the first one column for U as it is a latent



unique_rows, counts = np.unique(dataset, axis=0, return_counts=True)


joint_distribution = counts / len(dataset)

# Print the unique rows and their corresponding probabilities
for row, prob in zip(unique_rows, joint_distribution):
    print(f"Row: {row}, Probability: {prob:.4f}")



Row: [0 0 0], Probability: 0.0263
Row: [0 0 1], Probability: 0.0755
Row: [0 1 0], Probability: 0.1017
Row: [0 1 1], Probability: 0.0908
Row: [1 0 0], Probability: 0.0882
Row: [1 0 1], Probability: 0.4021
Row: [1 1 0], Probability: 0.1125
Row: [1 1 1], Probability: 0.1029


In [8]:
import numpy as np



# N=10000
# sampleZ = np.random.binomial(n=1, p=pz[id,1], size=N)
# sampleX_Z = np.random.binomial(n=1, p=px_z[id,1], size=N)
# sampleY_XZ = np.random.binomial(n=1, p=py_xz[id,1], size=N)

 

sampleX = torch.nn.functional.one_hot(torch.tensor(dataset[:,0]), num_classes=2).float()
sampleZ = torch.nn.functional.one_hot(torch.tensor(dataset[:,1]), num_classes=2).float()  
sampleY = torch.nn.functional.one_hot(torch.tensor(dataset[:,2]), num_classes=2).float()




# Training code

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim


def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape)
    return -torch.log(-torch.log(U + eps) + eps)


def gumbel_softmax_sample(logits, temperature, gumbel_noise):

    if gumbel_noise==None:
        gumbel_noise= sample_gumbel(logits.size())

    y = logits + gumbel_noise
    return F.softmax(y / temperature, dim=-1)


def gumbel_softmax(logits, temperature, gumbel_noise=None, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    output_dim =logits.shape[1]
    y = gumbel_softmax_sample(logits, temperature, gumbel_noise)

    if not hard:
        ret = y.view(-1, output_dim)
        return ret

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
    # ret = y_hard.view(-1, output_dim)
    ret = y_hard.view(-1, output_dim)
    return ret


class SampleGenerator(nn.Module):
    def __init__(self, input_size, output_size=2):
        super(SampleGenerator, self).__init__()
        hidden_size=50
        # Define learnable latent parameter (no input needed)
        self.latent = nn.Parameter(torch.randn(N, hidden_size))
        
        # if input_size==0:
        input_size= input_size+ hidden_size
        
        # Define the layers
        self.fc0 = nn.Linear(input_size, hidden_size)  # Hidden layer
        self.fc1 = nn.Linear(hidden_size, hidden_size)  # Hidden layer
        self.fc2 = nn.Linear(hidden_size, output_size)  # Output layer to predict logits

    def forward(self, parents, temperature=1.0, hard=False):
        

        if parents==None:
            parents=self.latent
        else:
            parents= torch.tensor(parents)
            parents= torch.cat([parents, self.latent], dim=1)
        
        # Use the learnable latent vector
        x= torch.relu(self.fc0(parents))
        x = torch.relu(self.fc1(x))
        logits = torch.relu(self.fc2(x))
                
        
        #
        # logits = logits / torch.max(logits)  # normalize logits
        #
        
        # Apply Gumbel-Softmax to generate differentiable one-hot like vectors
        # gumbel_softmax_output = nn.functional.gumbel_softmax(logits, tau=temperature, hard=hard, eps=1e-10)
        
        gumbel_softmax_output= gumbel_softmax(logits , temperature, None, False)
        
        
        output=gumbel_softmax_output
        if torch.isnan(torch.max(output)):
            print('--->',output.shape, ' logists-->', torch.max(logits),torch.min(logits), ' tau-->', temperature)
        
        
        return gumbel_softmax_output


In [10]:

def marginal(input, freq):
    
    if input==None:
        comb=freq
    else:
        # inp= torch.argmax(input, dim=1).view(-1,1)
        comb= torch.cat([freq, input], dim=1)
            
    unique_rows, counts = np.unique(np.array(comb), axis=0, return_counts=True)
    joint_distribution = counts / len(dataset)
    
    rdict={}
    for row, prob in zip(unique_rows, joint_distribution):
        rdict[tuple(row)]= prob
        # print(f"Row: {row}, Probability: {prob:.4f}")
    
    return rdict




def train(input, targets, model):

    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop (for one sample per forward pass)
    for epoch in range(2000):
        optimizer.zero_grad()  # Zero the gradients
    
        # Forward pass to generate sample
        output = model(input, temperature=1.0, hard=False)  # Model generates sample
    
        # Compute loss
        loss = criterion(output, targets) #P(Y|X,Z)
        
        if torch.isnan(loss):
            print(torch.max(output), torch.min(output))
            break
            

        
        # Backpropagation and optimizer step
        loss.backward()
        optimizer.step()
    
        if (epoch+1) % 50 == 0:
            
            freq= torch.argmax(output, dim=1).view(-1,1)
            rdict= marginal(input, freq)
            
            
            freq= torch.argmax(targets, dim=1).view(-1,1)
            tdict= marginal(input, freq)
                
            tvd=0
            for key in rdict:
                tvd+= abs(rdict[key]- tdict[key])
                


            ###
            
            print(f'Epoch [{epoch+1}/1000], Loss: {loss.item():.4f} obsTVD:{tvd/2:0.4f}')
            
    
    print("Training complete!")
    
    return model

# Train P(Y|X,Z)

In [17]:
input= torch.cat([sampleX, sampleZ], dim=1)
targets = sampleY
input_size=4

modelY = SampleGenerator(input_size=input_size)  # P(Y|X,Z)
modelY= train(input, targets, modelY)

/tmp/ipykernel_1104894/3770493806.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  parents= torch.tensor(parents)


Epoch [50/1000], Loss: 0.6763 obsTVD:0.1331
Epoch [100/1000], Loss: 0.5927 obsTVD:0.2335
Epoch [150/1000], Loss: 0.4443 obsTVD:0.0596
Epoch [200/1000], Loss: 0.3792 obsTVD:0.0377
Epoch [250/1000], Loss: 0.3665 obsTVD:0.0329
Epoch [300/1000], Loss: 0.3630 obsTVD:0.0300
Epoch [350/1000], Loss: 0.3608 obsTVD:0.0287
Epoch [400/1000], Loss: 0.3598 obsTVD:0.0279
Epoch [450/1000], Loss: 0.3593 obsTVD:0.0272
Epoch [500/1000], Loss: 0.3586 obsTVD:0.0266
Epoch [550/1000], Loss: 0.3579 obsTVD:0.0260
Epoch [600/1000], Loss: 0.3577 obsTVD:0.0258
Epoch [650/1000], Loss: 0.3574 obsTVD:0.0255
Epoch [700/1000], Loss: 0.3572 obsTVD:0.0253
Epoch [750/1000], Loss: 0.3572 obsTVD:0.0253
Epoch [800/1000], Loss: 0.3571 obsTVD:0.0254
Epoch [850/1000], Loss: 0.3570 obsTVD:0.0253
Epoch [900/1000], Loss: 0.3568 obsTVD:0.0251
Epoch [950/1000], Loss: 0.3565 obsTVD:0.0248
Epoch [1000/1000], Loss: 0.3560 obsTVD:0.0247
Epoch [1050/1000], Loss: 0.3560 obsTVD:0.0247
Epoch [1100/1000], Loss: 0.3558 obsTVD:0.0245
Epoch [1

# Train P(Z|X)




In [18]:
input= torch.cat([sampleX], dim=1)
targets = sampleZ
input_size=2

modelZ = SampleGenerator(input_size=input_size)  # P(Z|X)
modelZ= train(input, targets, modelZ)

/tmp/ipykernel_1104894/3770493806.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  parents= torch.tensor(parents)


Epoch [50/1000], Loss: 0.6763 obsTVD:0.2349
Epoch [100/1000], Loss: 0.5153 obsTVD:0.0584
Epoch [150/1000], Loss: 0.4081 obsTVD:0.0441
Epoch [200/1000], Loss: 0.3802 obsTVD:0.0362
Epoch [250/1000], Loss: 0.3736 obsTVD:0.0326
Epoch [300/1000], Loss: 0.3705 obsTVD:0.0320
Epoch [350/1000], Loss: 0.3680 obsTVD:0.0301
Epoch [400/1000], Loss: 0.3668 obsTVD:0.0294
Epoch [450/1000], Loss: 0.3660 obsTVD:0.0291
Epoch [500/1000], Loss: 0.3649 obsTVD:0.0280
Epoch [550/1000], Loss: 0.3648 obsTVD:0.0279
Epoch [600/1000], Loss: 0.3642 obsTVD:0.0279
Epoch [650/1000], Loss: 0.3636 obsTVD:0.0274
Epoch [700/1000], Loss: 0.3629 obsTVD:0.0268
Epoch [750/1000], Loss: 0.3627 obsTVD:0.0268
Epoch [800/1000], Loss: 0.3625 obsTVD:0.0268
Epoch [850/1000], Loss: 0.3622 obsTVD:0.0269
Epoch [900/1000], Loss: 0.3619 obsTVD:0.0268
Epoch [950/1000], Loss: 0.3619 obsTVD:0.0268
Epoch [1000/1000], Loss: 0.3618 obsTVD:0.0267
Epoch [1050/1000], Loss: 0.3618 obsTVD:0.0267
Epoch [1100/1000], Loss: 0.3615 obsTVD:0.0264
Epoch [1

# Inferene

In [19]:
intval=1
intval= intval*torch.ones(N).to(torch.int64)
outX= torch.nn.functional.one_hot(intval, num_classes=2).float()

outZ = modelZ(outX, temperature=1.0, hard=True)

outY= modelY(torch.cat([sampleX, outZ], dim=1), temperature=1.0, hard=True)


/tmp/ipykernel_1104894/3770493806.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  parents= torch.tensor(parents)


In [20]:

outt= outY
outt= torch.argmax(outt, dim=1).view(-1,1)
unique_rows, counts = np.unique(np.array(outt.detach()), axis=0, return_counts=True)
joint_distribution = counts / len(outt)
joint_distribution

array([0.3049, 0.6951])